In [1]:
from hist import Hist, axis, NamedHist
import boost_histogram as bh

In [2]:
h = Hist(
    axis.Regular(10, 0, 1)
)

In [3]:
h.fill([1,2,3])

Hist(Regular(10, 0, 1, metadata={'name': None}), storage=Double()) # Sum: 0.0 (3.0 with flow)

In [4]:
h2 = h + h

In [5]:
h.view(flow=True)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 3.])

In [6]:
h2.view(flow=True)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 6.])

# NamedHist is a type of Histogram that requires axes with a name
### 1. Each axis should have a name
### 2. Filling should be done using keywords as axis names
### 3. Axis names could also be used for indexing

## Passing an axis without a name would result in an error

In [7]:
h_named = NamedHist(
    axis.Regular(10, 0, 1)
)

KeyError: 'The name of axis required'

In [8]:
h_named = NamedHist(
    axis.Regular(10, 0, 1, name='x'),
    axis.Regular(10, 0, 1, name='y')
)

## Filling can be done using axis names as keywords

In [9]:
h_named.fill(x=[0.45, 0.65, 0.75], y=[0.55, 0.45, 0.65])
h_named.view()

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

## Filling without using keywords results in an error

In [10]:
h_named.fill([0.45, 0.65, 0.75], [0.55, 0.45, 0.65])

ValueError: Filling should be done using keyword arguments only

## Axis names can be used for indexing

In [11]:
h_named = NamedHist(
    axis.Regular(5, 0, 1, name='x'),
    axis.Regular(5, 0, 1, name='y')
)
h_named.fill(x=[0.3, 0.5, 0.7], y=[0.1, 0.3, 0.5])

h_named.view()

array([[0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0.]])

### The following lines are equivalent

In [12]:
h2 = h_named[1:4, :]
h3 = h_named[{0 : slice(1, 4, None)}]
h4 = h_named[{'x': slice(1, 4, None)}]

In [13]:
h2.view()

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.]])

In [14]:
h3.view()

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.]])

In [15]:
h4.view()

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.]])

### Further slicing can also be done using the same way

In [16]:
h5 = h4[{'y': slice(None, 3, None)}]
h5.view()

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

### The following lines are equivalent

In [17]:
h2 = h_named[:, ::bh.sum]
h3 = h_named[{1: slice(None, None, bh.sum)}]
h4 = h_named[{'y': slice(None, None, bh.sum)}]

In [18]:
h2.view()

array([0., 1., 1., 1., 0.])

In [19]:
h3.view()

array([0., 1., 1., 1., 0.])

In [20]:
h4.view()

array([0., 1., 1., 1., 0.])

## There is a bug that raises an error when using bh.loc() in a dict

In [21]:
hst = bh.Histogram(
    axis.Regular(5, 0, 5),
    axis.Regular(5, 0, 5)
)

hst.fill([1.5, 2.5], [0.5, 3.5])
hst.view()

array([[0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

### The following line will give an error

In [22]:
h2 = hst[{0: bh.loc(2.5)}]

IndexError: Must be a slice, an integer, or follow the locator protocol.

## The BaseHist class has been implemented to fix the error

In [23]:
hst = Hist(
    axis.Regular(5, 0, 5),
    axis.Regular(5, 0, 5)
)
hst.fill([1.5, 2.5], [0.5, 3.5])
hst.view()

array([[0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

### No error is raised

In [24]:
h2 = hst[{0: bh.loc(2.5)}]
h2.view()

array([0., 0., 0., 1., 0.])

### The same could be achieved using NamedHist

In [25]:
hst = NamedHist(
    axis.Regular(5, 0, 5, name='x'),
    axis.Regular(5, 0, 5, name='y')
)
hst.fill(x=[1.5, 2.5], y=[0.5, 3.5])

h2 = hst[{'x': bh.loc(2.5)}]
h2.view()

array([0., 0., 0., 1., 0.])

# bool is a shortcut for an Integer axis with no underflow or overflow and two bins starting from zero

In [26]:
h = Hist(
    axis.bool()
)

h.view()

array([0., 0.])

In [27]:
h.fill([0, 0, 1])
h.view()

array([2., 1.])

### There are no flow bins

In [28]:
h.view(flow=True)

array([2., 1.])

### Using bool with NamedHist

In [29]:
h = NamedHist(
    axis.Regular(5, 0, 5, name='x'),
    axis.bool(name='valid')
)
h.view()

array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]])

In [30]:
h.fill(x=[2, 3, 4], valid=[0, 0, 1])
h.view()

array([[0., 0.],
       [0., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.]])